In [1]:
import pandas as pd
import numpy_financial as npf
from datetime import date, datetime
import plotly.express as px

In [2]:
interest = 0.04
years = 30
payments_year = 12
mortgage = 400000
start_date = (date(2022, 5, 1))

# SCENARIO #1
# General: Same payment each month until loan is paid off
extra_payment = 300

# SCENARIO #2
# Investor/Landlord: Dynamic payment based on % of rental cash flow
start_rent = None
start_cash_flow = None
rent_increase_yoy = None
extra_payment_prct = None

In [3]:
  initial_pmt = -1 * npf.pmt(interest/12, years*payments_year, mortgage)
  initial_ipmt = -1 * npf.ipmt(interest/payments_year, 1, years*payments_year, mortgage)
  initial_ppmt = -1 * npf.ppmt(interest/payments_year, 1, years*payments_year, mortgage)

  # create dataframe
  rng = pd.date_range(start_date, periods=years * payments_year, freq='MS')
  rng.name = "Payment Date"
  initial_cols_list = ['Org Total Payment','Total Payment','Interest','Principal',
                  'Rent','Cash Flow','Additional Payment','Org Ending Balance','Ending Balance']
  df = pd.DataFrame(index=rng, columns=initial_cols_list, dtype='float')
  df.reset_index(inplace=True)
  df.index += 1
  df.index.name = "Period"

  # test if additional cash flow from rent is passed as an arg
  if (start_rent !=None) and (start_cash_flow != None) and (rent_increase_yoy != None) and (extra_payment_prct !=None):
    initial_additional_pmt = start_cash_flow * extra_payment_prct
  elif extra_payment != None:
    initial_additional_pmt = extra_payment
  else:
    initial_additional_pmt = 0


  # create first row
  period = 1
  initial_row_dict = {
    'Org Total Payment': initial_pmt,
    'Total Payment': initial_pmt + (initial_additional_pmt),
    'Interest': initial_ipmt,
    'Principal': initial_ppmt,
    'Rent': start_rent,
    'Cash Flow': start_cash_flow,
    'Additional Payment': initial_additional_pmt,
    'Org Ending Balance': mortgage - initial_ppmt,
    'Ending Balance': mortgage - initial_ppmt - (initial_additional_pmt)
  }
  columns = list(initial_row_dict.keys())
  period_values = list(initial_row_dict.values())
  df.at[period, columns]=period_values
  df = df.round(2)

  # add additional rows
  for period in range(2, len(df) + 1):
    # get prior period values
    previous_total_payment = df.loc[period - 1, 'Total Payment']
    previous_principal = df.loc[period - 1, 'Principal']
    previous_rent = df.loc[period - 1, 'Rent']
    previous_cf = df.loc[period - 1, 'Cash Flow']
    previous_org_ending_balance = df.loc[period - 1, 'Org Ending Balance']
    previous_ending_balance = df.loc[period - 1, 'Ending Balance']
    
    # get additional payment values
    if (start_rent !=None) and (start_cash_flow != None) and (rent_increase_yoy != None) and (extra_payment_prct !=None):
      if period % 13 == 0:
        period_rent = previous_rent * (1 + rent_increase_yoy)
      else:
        period_rent = previous_rent
      period_cash_flow = previous_cf + (period_rent - previous_rent)
      period_additional_pmt = period_cash_flow * extra_payment_prct
    elif extra_payment != None:
      period_additional_pmt = initial_additional_pmt
      period_rent = 0
      period_cash_flow = 0
      extra_payment_prct = 0
    else:
      period_additional_pmt = 0
      period_rent = 0
      period_cash_flow = 0
      extra_payment_prct = 0

    # get end balance
    period_interest = previous_org_ending_balance * interest / payments_year
    period_principal = initial_pmt - period_interest
    org_ending_balance = previous_org_ending_balance - period_principal
    ending_balance = previous_ending_balance - period_principal - period_additional_pmt
    org_ending_balance = 0 if org_ending_balance <= 0 else org_ending_balance
    ending_balance = 0 if ending_balance <= 0 else ending_balance


    row_dict = {'Org Total Payment': initial_pmt,
                'Total Payment': initial_pmt + (period_cash_flow * extra_payment_prct),
                'Interest': period_interest,
                'Principal': period_principal,
                'Rent': period_rent,
                'Cash Flow': period_cash_flow,
                'Additional Payment': period_additional_pmt,
                'Org Ending Balance': org_ending_balance,
                'Ending Balance': ending_balance}
    columns = list(row_dict.keys())
    period_values = list(row_dict.values())
    df.at[period,columns]=period_values

TypeError: unhashable type: 'list'